In [ ]:
#Loads necessary libraries for data analysis and visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    roc_auc_score, roc_curve, accuracy_score
)
import warnings
warnings.filterwarnings("ignore")

plt.style.use('seaborn-v0_8')

In [ ]:
# Load processed data (small parquet samples to avoid lag)
alerts = pd.read_parquet("data/processed/synthetic_alerts_sample.parquet")
transactions = pd.read_parquet("data/processed/synthetic_transactions_sample.parquet")

print(f"Data loaded successfully: {alerts.shape[0]} alerts, {transactions.shape[0]} transactions.\n")
print("Alerts dataset shape:", alerts.shape)
print("Transactions dataset shape:", transactions.shape)

alerts.head()

In [ ]:
# Merge transactions and alerts by AlertID
#this is an inner join to ensure only transactions with corresponding alerts are included
merged = transactions.merge(alerts, on="AlertID", how="inner")

print("Merged dataset shape:", merged.shape)
merged.head()

In [ ]:
# Convert text labels to binary format
# 'Report' = 1 (suspicious), 'Dismiss' = 0 (normal)
merged['Label'] = merged['Outcome'].map({'Report': 1, 'Dismiss': 0})

# Basic cleanup and feature selection
X = merged[['Size']]
y = merged['Label']

print(X.head())
print(y.value_counts())


In [ ]:
# If you have a saved model:
# from joblib import load
# model = load('results/models/aml_model.joblib')
# y_pred = model.predict(X)

# TEMP: Simulate predictions using random probability
np.random.seed(42)
merged['Predicted_Prob'] = np.random.rand(len(merged))
merged['Predicted_Label'] = (merged['Predicted_Prob'] > 0.5).astype(int)

y_pred = merged['Predicted_Label']
y_prob = merged['Predicted_Prob']
#this simulates model predictions using random probabilities

In [ ]:
#this evaluates model performance using various metrics and numerically shows how good the model is at detecting suspicious activities
print("Classification Report:")
print(classification_report(y, y_pred, target_names=["Dismiss", "Report"]))

print("Accuracy:", accuracy_score(y, y_pred))
print("ROC AUC:", roc_auc_score(y, y_prob))

In [ ]:
#builds a confusion matrix visualizing the performance of the classification model using a heatmap 
cm = confusion_matrix(y, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=["Pred Dismiss", "Pred Report"],
            yticklabels=["True Dismiss", "True Report"])
plt.title("Confusion Matrix")
plt.show()
#helps identify where the model is making correct and incorrect predictions

In [ ]:
#this builds and visualizes a ROC curve to evaluate model performance showing the trade-off between true positive rate and false positive rate
#shows AUC score indicating model's ability to distinguish between classes. A higher AUC indicates better performance
fpr, tpr, _ = roc_curve(y, y_prob)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y, y_prob):.2f}")
plt.plot([0,1], [0,1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

In [ ]:
#this summarizes key metrics and provides insights for understanding of model performance
print(f"Evaluation complete.\n")
print(f"Accuracy: {accuracy_score(y, y_pred):.3f}")
print(f"AUC: {roc_auc_score(y, y_prob):.3f}")
print("\nInsights:")
print("- 'Report' represents suspicious alerts flagged by the system.")
print("- Focus on reducing False Negatives (missed suspicious activities).")
print("- If model predicts too many 'Reports', precision drops — adjust thresholds.")